# App Layout & Callbacks

In the previous notebook (`00_project_startup.ipynb`), we prepared almost all the backend logic:
* We loaded the data and organized it by tab
* We computed KPI values for our stat cards
* We set up dictionaries to map tabs to builder functions
* We defined reusable style and layout helpers

In this notebook, we now bring everything **together in the actual Dash app**.  
We'll walk through the full structure of app.py, focusing on two key elements:

1. **The Layout** – How we build the visual page using Dash + Bootstrap components (e.g., `dbc.Container`, `dcc.Tabs`, `dbc.Row`, and `dbc.Col`)
2. **The Callback Logic** – How tab selections dynamically control what graphs are shown, and how we route those through helper functions and builders in a clean, scalable way.

We'll cover:
* How the layout is structured using Dash and Bootstrap (logo, KPI cards, tab bars, and figures)
* How a single `@app.callback` connects the selected tabs to the correct data and visualisations
* How to add new tabs or views by updating mappings without changing core logic

We’ll go through the layout row by row, and then explain the callback that powers the dynamic behaviour.


---
## App Layout

We'll start by initializing the dash app as follows:

In [6]:
from pathlib import Path

import dash_bootstrap_components as dbc
import pandas as pd
from dash import Dash, dcc, html, Input, Output  # State not currently needed

from src.const import get_constants
from src import dash1, dash2, dash3, dash4

# ──────────────────────────────────────────────────────────────────────────────
# Data & constants
# ──────────────────────────────────────────────────────────────────────────────
DATA_DIR = Path("./data")

MOVIES = pd.read_csv(DATA_DIR / "movie_after_cleaning.csv")
MOVIES_SPLITS = pd.read_excel(DATA_DIR / "splits_movie.xlsx", sheet_name=None)
SERIES = pd.read_csv(DATA_DIR / "series_after_cleaning.csv")
SERIES_SPLITS = pd.read_excel(DATA_DIR / "splits_series.xlsx", sheet_name=None)

DATA_BY_TAB = {
    "movie": (MOVIES, MOVIES_SPLITS),
    "series": (SERIES, SERIES_SPLITS),
}

VISUALIZATION_BUILDERS = {
    "overview": (dash1.generate_visualizations, 4),
    "content_creators": (dash2.generate_visualizations, 4),
    "parental": (dash3.generate_visualizations, 2),
    "year": (dash4.generate_visualizations, 2),
}

# Top-level stats
NUM_WORKS, NUM_COUNTRIES, NUM_LANGUAGES, AVG_VOTES = get_constants(
    MOVIES, SERIES, MOVIES_SPLITS, SERIES_SPLITS
)

MAX_OPTIONS_DISPLAY = 3_300
DROPDOWN_OPTIONS = {
    "movie": [{"label": t, "value": t} for t in MOVIES["title"][:MAX_OPTIONS_DISPLAY]],
    "series": [{"label": t, "value": t} for t in SERIES["title"][:MAX_OPTIONS_DISPLAY]],
}

BRAND_COLOR = "#deb522"

CARD_STYLE = {
    "paddingBlock": "10px",
    "backgroundColor": BRAND_COLOR,
    "border": "none",
    "borderRadius": "10px",
}

TAB_STYLE_IDLE = {
    "borderRadius": "10px",
    "padding": 0,
    "marginInline": "5px",
    "display": "flex",
    "alignItems": "center",
    "justifyContent": "center",
    "fontWeight": "bold",
    "backgroundColor": BRAND_COLOR,
    "border": "none",
}
TAB_STYLE_ACTIVE = {**TAB_STYLE_IDLE, "textDecoration": "underline"}

# ──────────────────────────────────────────────────────────────────────────────
# Helpers
# ──────────────────────────────────────────────────────────────────────────────
def stats_card(title: str, value, img: str) -> html.Div:
    """Single KPI card."""
    return html.Div(
        dbc.Card(
            [
                dbc.CardImg(src=img, top=True, style={"width": "50px", "alignSelf": "center"}),
                dbc.CardBody(
                    [
                        html.P(value, style={"margin": 0, "fontSize": "22px", "fontWeight": "bold"}),
                        html.H4(title, style={"margin": 0, "fontSize": "18px", "fontWeight": "bold"}),
                    ],
                    style={"textAlign": "center"},
                ),
            ],
            style=CARD_STYLE,
        )
    )


def wrap_figures(figures) -> html.Div:
    """Lay out a list of Plotly figures in a 2-column grid."""
    return html.Div(
        [
            html.Div(dcc.Graph(figure=fig), style={"width": "50%", "display": "inline-block"})
            for fig in figures
        ]
    )


In [ ]:
app = Dash(
    __name__,  # standard Python name reference
    external_stylesheets=[dbc.themes.BOOTSTRAP],  # loads the Bootstrap styles
    title="IMDB Data Analysis Dashboard"  # sets the browser tab title
)

Dash(...) creates the main app object. then, 
```external_stylesheets=[dbc.themes.BOOTSTRAP]``` links to a prebuilt Bootstrap CSS theme. (You can change this to other themes like dbc.themes.CYBORG or dbc.themes.SLATE, you can find more online, like here: https://startbootstrap.com/themes)

title=... sets what the browser tab says when the app is running.

Then, we start the bulk of the work in this part, the app.layout.Here, you will insert all of the containers an tabs that you want to have in your final dashboard:
```python
# Define the layout of the entire page
app.layout = html.Div(
    ...
)
```

Here, the layout is what Dash uses to render the app's HTML structure.

html.Div is a basic container, everything visible to the user goes inside this container. And all the contents are stacked vertically (top to bottom), unless styled differently.

We then open a dbc.container:
```python
dbc.Container([
    ...
], style={"padding": 0})
```


A dbc container is part of Dash Bootstrap Components (dbc). It gives you a responsive grid layout that adjusts to screen sizes and it aligns content nicely on desktop and mobile. The inner layout will follow a 12-column Bootstrap grid (explained more in later cells). In this example, we remove internal padding using style={"padding": 0} for full-width edge-to-edge layout. But you may use padding if you like it!



---
### dbc.Row and dbc.Col (The Core Grid System)

Now, lets walk through the rows and columns one by one:

#### Row 1 (Header)
```python
dbc.Row([
    dbc.Col(html.Img(src="./assets/imdb.png", width=150), width=2),
    dbc.Col(
        dcc.Tabs(...),  # Graph tab navigation
        width=6,
    ),
])
```
As is intuitive,
dbc.Row creates a horizontal row and dbc.Col defines columns within that row.

In this header, We give the logo 2/12 columns and the tab bar 6/12 columns of space. When we run the app, it comes out as is shown in the picture below. I annotated it, so you can see how space is taken up in the header.

![Alt text](assets/header_annot.png)

This is an important Bootstrap concept to remember: The entire row always spans 12 columns. Also, columns will wrap automatically on small screens. And finally, you can omit width= to let columns auto-size, or specify exact width=N.

Inside of this logic is then the bulk of what the user will actually see. In this case, this is shown in the dcc.Tabs, as we want to use tabs to navigate our dashboard:

In [8]:
dcc.Tabs(
    id="graph-tabs",
    value="overview",
    children=[
        dcc.Tab(label="Overview", value="overview", style=TAB_STYLE_IDLE, selected_style=TAB_STYLE_ACTIVE),
        dcc.Tab(label="Content creators", value="content_creators", style=TAB_STYLE_IDLE, selected_style=TAB_STYLE_ACTIVE),
        dcc.Tab(label="Parental Guide", value="parental", style=TAB_STYLE_IDLE, selected_style=TAB_STYLE_ACTIVE),
        dcc.Tab(label="Year", value="year", style=TAB_STYLE_IDLE, selected_style=TAB_STYLE_ACTIVE),
    ],
    style={"marginTop": "15px", "width": "600px", "height": "50px"},
)

Tabs(children=[Tab(label='Overview', selected_style={'borderRadius': '10px', 'padding': 0, 'marginInline': '5px', 'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center', 'fontWeight': 'bold', 'backgroundColor': '#deb522', 'border': 'none', 'textDecoration': 'underline'}, style={'borderRadius': '10px', 'padding': 0, 'marginInline': '5px', 'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center', 'fontWeight': 'bold', 'backgroundColor': '#deb522', 'border': 'none'}, value='overview'), Tab(label='Content creators', selected_style={'borderRadius': '10px', 'padding': 0, 'marginInline': '5px', 'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center', 'fontWeight': 'bold', 'backgroundColor': '#deb522', 'border': 'none', 'textDecoration': 'underline'}, style={'borderRadius': '10px', 'padding': 0, 'marginInline': '5px', 'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center', 'fontWeight': 'bold', 'backgroundColor': '#deb522', 'border': 'no

As said above, this creates a horizontal navigation tab bar with four tabs: Overview,  Content creators, Parental Guide, and Year

Each of these tabs corresponds to a different "view" or visualization in the dashboard.

**Explanation of Key Properties**:

| Property                          | Description                                                                               |
| --------------------------------- | ----------------------------------------------------------------------------------------- |
| `id="graph-tabs"`                 | Unique ID for this tab component. Used in callbacks.                                      |
| `value="overview"`                | Sets the default selected tab when the app loads.                                         |
| `children=[...]`                  | Defines the tab items. Each tab has a `label` (shown in UI) and a `value` (used in code). |
| `style=TAB_STYLE_IDLE`            | Styling applied to unselected tabs.                                                       |
| `selected_style=TAB_STYLE_ACTIVE` | Styling for the currently selected tab (e.g., underline).                                 |
| `style={...}`                     | Applies styling to the entire tab bar (e.g., spacing and width).                          |

---

We see that we now use those styles like "TAB_STYLE_ACTIVE", as we defined in the previous notebook, to easily access predefined styles for our dashboard.

Using a consistent structure like this, with `dcc.Tab(...)` and predefined styles allows:

* Clear visual feedback on which tab is selected
* A single callback to handle all logic based on tab values
* Easy extension — just add another `dcc.Tab(...)` to introduce a new view

This approach keeps the layout modular and avoids hardcoding logic multiple times.

#### Row 2 (KPI Cards)
As shown in the dashboard picture above, the large yellow cards at the top, show important metrics (KPI values) that we calculated earlier using helper functions (`const.py`). Now that we've defined the `stats_card` function in the previous notebook, we can reuse it here to easily generate each KPI card by passing in the corresponding value, such as `NUM_WORKS`, `NUM_LANGUAGES`, and so on. This keeps the layout clean and the code reusable.



In [ ]:
dbc.Row(
[
    dbc.Col(stats_card("Work", NUM_WORKS, "./assets/movie-icon.png"), width=3),
    dbc.Col(stats_card("Language", NUM_LANGUAGES, "./assets/language-icon.svg"), width=3),
    dbc.Col(stats_card("Country", NUM_COUNTRIES, "./assets/country-icon.png"), width=3),
    dbc.Col(stats_card("Average Votes", AVG_VOTES, "./assets/vote-icon.png"), width=3),
],
style={"marginBlock": "10px"},
)

Again, using the same logic as before, each column takes 3/12 so that 4 cards fit side-by-side in a single row. marginBlock: 10px adds spacing above and below this row, so that the header and this row aren't aligning against each other. We also add a picture to identify the type of KPI metric shown, simply by adding the .png from your directory.

#### Row 3 (Movie/Series Tabs)

One important selling point of this use case, is the fact that we can easily switch between viewing the data on movies and the data on series, using the same viewings as in the top tabs. This way, we have 4 views (Overview, Content Creators,...), but we also have 2 different "datasets" that these views can take their data from (Movies and Series). In this way, we literally multiply the options in our dashboard to 4x2 = 8!

The overall structure of this third row is simple and very similar to the previous ones:
```python
dbc.Row(
    dcc.Tabs(
        id="data-tabs",
        value="movie",
        children=[
            dcc.Tab(...),
            dcc.Tab(...),
        ],
        style={"padding": 0}
    )
)
```

Again, the dcc.Tabs holds multiple dcc.Tab items. The value="movie" defines which tab is selected by default. Each Tab has a style and a selected_style to define appearance, we defined the main yellow color before, as you know (the BRAND_COLOR). We customized the styling to fit our dark background and brand yellow color. The styling after "selected_style might seem daunting, but its simply explained by the terminology of the style keys (which you can find online as well, if you want to learn more of them):

| Key                 | Purpose                                                               |
| ------------------- | --------------------------------------------------------------------- |
| `"border"`          | Draws a white border around the tab to make it stand out.             |
| `"backgroundColor"` | Matches the black background of the dashboard.                        |
| `"color"`           | Uses the yellow brand color for text.                                 |
| `"fontWeight"`      | Makes the label bold for readability.                                 |
| `"textDecoration"`  | Adds an underline when selected to visually show which tab is active. |




#### Row 4 (Dynamic Content Area)
Alright, now for the most important area, the visualizations! I'll call ths the "dynamic content area", as this is where the visualizations will show up, but also where the callbacks will update the visualizations based on the tabs we have selected. The code is fairly simple, just something like:

In [ ]:
dbc.Row(
    dcc.Loading(html.Div(id="tabs-content"), type="default", color=BRAND_COLOR)
)

* The html.Div(id="tabs-content") is updated dynamically by a Dash callback.
* The dcc.Loading(...) wrapper shows a spinner while the graphs are loading. Just a nice aeshetic touch :)

Lastly, the full outermost html.Div wraps the entire app layout:
```python
html.Div(..., style={"backgroundColor": "black", "minHeight": "100vh"})
```
We set a black background and minHeight: 100vh so it always fills the screen vertically.


---

## Callbacks

Now that we have the layout in place, we connect it all together using a single Dash callback. You can use multiple callbacks in your project, one per section of your dashboard, or per view, or per tab. But for simplicity, we use one for everything here, keeping a clean strucutre. This callback listens to both sets of tabs:
- graph-tabs (which visualization to show, these are the tabs in the header)
- data-tabs (whether to use movies or series data)

This means: every time you change a tab, this function is triggered to recompute the correct figures and update the dashboard.

Let's take a look at the code, its quite short, but there's a lot going on!:



In [ ]:
@app.callback(
    Output("tabs-content", "children"),  # Where the new layout goes
    Input("graph-tabs", "value"),        # Which visual to show (overview, creators, etc.)
    Input("data-tabs", "value"),         # Which dataset to use (movie or series)
)
def update_tab(graph_tab: str, data_tab: str):
    """Render the correct set of figures based on tab selections."""

    # 1 Load the correct dataset based on Movie / Series
    data, splits = DATA_BY_TAB[data_tab]

    # 2 Get the correct visualisation builder function
    builder, expected_figs = VISUALIZATION_BUILDERS[graph_tab]

    # 3 Generate the figures
    figures = builder(data, splits)

    # 4 Fail early if the builder returns the wrong number of figures
    if len(figures) != expected_figs:
        raise ValueError(f"{builder.__name__} returned {len(figures)} figures (expected {expected_figs}).")

    # 5 Wrap and return them as Dash layout components
    return wrap_figures(figures)


So what's really going on here, in simple terms?

Think of the callback as a **smart traffic controller**. Whenever you click a new tab — whether you're switching from "Overview" to "Parental Guide", or from "Movies" to "Series" — Dash sends that signal (the value of the selected tabs) into this one function.

From there:

**1. We grab the correct data.**
Instead of writing a bunch of `if tab == ...` statements, we use the dictionary we defined in the previous notebokok as:

```python
data, splits = DATA_BY_TAB[data_tab]
```

So when the user selects "movie", this line says:

_“Hey, what data do I need for 'movie'?”_

It gives us two DataFrames:

* One with the full dataset (e.g. all movies)
* One with the pre-processed “splits” (e.g. countries, languages, etc.)<br><br>

**2. We ask another dictionary which function to use.**
For the visualizations, we don't hardcode logic. We just ask:

```python
builder, expected_figs = VISUALIZATION_BUILDERS[graph_tab]
```

_“If the tab is 'overview', which function should I run to build the graphs?”_

That gives us something like `dash1.generate_visualizations` for `builder`.

Also, remember from the **previous notebook**, we defined this dictionary:

```python
VISUALIZATION_BUILDERS = {
    "overview": (dash1.generate_visualizations, 4),
    "content_creators": (dash2.generate_visualizations, 4),
    "parental": (dash3.generate_visualizations, 2),
    "year": (dash4.generate_visualizations, 2),
}
```

Each key here corresponds to one of the top-level graph-tabs in the app.
Each value is a tuple:

* A function (like `dash3.generate_visualizations`) that builds that page’s plots
* The number of expected figures (to help us catch errors)

These functions live in `src/dash1.py`, `dash2.py`, etc.
But don’t worry — these files are nothing more than **plain Plotly visualizations**, wrapped inside a function. Each one takes `(data, splits)` and returns a list of Plotly `figure` objects.

**We’ll go into full detail on writing those visualizations in the *next notebook***.<br><br>

**3. We run that function with the correct data.**
This executes the builder logic:

```python
figures = builder(data, splits)
```

Now we’ve got our list of Plotly figures.<br><br>

**4. We check the output — early.**
This line:

```python
if len(figures) != expected_figs:
    raise ValueError(...)
```

Helps us catch mistakes *before* they break the layout or crash the browser.
If your function was supposed to return 4 graphs but only gave 2, this will stop you immediately with a clear error message.<br><br>

**5. We lay out the graphs on the page.**
Lastly, we use the other helper function we defined:

```python
return wrap_figures(figures)
```

Which takes the Plotly figures and formats them in a 2-column layout using `dcc.Graph`.

---

### Why This Works So Well

- **It's clean.** No repetitive `if tab ==` spaghetti.
- **It's extensible.** Add a new view? Just define a new builder and register it in the dictionary.
- **It's maintainable.** Layout, logic, and styling are clearly separated.
- **It's fast.** Only the selected tab’s visualizations are computed.
- **It’s declarative.** All behavior flows from a simple set of mappings and helper functions.

So using this structure for your own project just makes your live easier!

